# [Линейная регрессия: прогноз оклада по описанию вакансии](https://www.coursera.org/learn/vvedenie-mashinnoe-obuchenie/programming/QFvJY/linieinaia-rieghriessiia-proghnoz-oklada-po-opisaniiu-vakansii)

## Введение

Линейные методы хорошо подходят для работы с разреженными данными — к таковым относятся, например, тексты. Это можно объяснить высокой скоростью обучения и небольшим количеством параметров, благодаря чему удается избежать переобучения.

Линейная регрессия имеет несколько разновидностей в зависимости от того, какой регуляризатор используется. Мы будем работать с гребневой регрессией, где применяется квадратичный, или L2-регуляризатор.

## 1. Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv (либо его заархивированную версию salary-train.zip).

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
data = pd.read_csv('./data/9_salary-train.csv')
data.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


## 2. Проведите предобработку:
* Приведите тексты к нижнему регистру (text.lower()).

In [2]:
data['FullDescription'] = data['FullDescription'].str.lower()
data['LocationNormalized'] = data['LocationNormalized'].str.lower()
data['ContractTime'] = data['ContractTime'].str.lower()
data.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london ****k ****...,london,permanent,33000
1,an ideal opportunity for an individual that ha...,london,permanent,50000
2,online content and brand manager// luxury reta...,south east london,permanent,40000
3,a great local marketleader is seeking a perman...,dereham,permanent,22500
4,registered nurse / rgn nursing home for young...,sutton coldfield,NaN,20355


* Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова. Для такой замены в строке text подходит следующий вызов: re.sub('[^a-zA-Z0-9]', ' ', text). Также можно воспользоваться методом replace у DataFrame, чтобы сразу преобразовать все тексты:

In [3]:
data = data.replace('[^a-zA-Z0-9]', ' ', regex=True)
data.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,london,permanent,33000
1,an ideal opportunity for an individual that ha...,london,permanent,50000
2,online content and brand manager luxury reta...,south east london,permanent,40000
3,a great local marketleader is seeking a perman...,dereham,permanent,22500
4,registered nurse rgn nursing home for young...,sutton coldfield,NaN,20355


* Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=5)
X = vectorizer.fit_transform(data['FullDescription'])

* Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'. Код для этого был приведен выше.

In [5]:
data['LocationNormalized'].fillna('nan', inplace=True)
data['ContractTime'].fillna('nan', inplace=True)

* Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.

In [6]:
from sklearn.feature_extraction import DictVectorizer
enc = DictVectorizer()
a = enc.fit_transform(data[['LocationNormalized', 'ContractTime']].to_dict('records'))

# Сильно разряженая матрица из всех столюцоы только два не нулехвых 
# так как в столбцах 'LocationNormalized', 'ContractTime' всего 2 значения
for i in np.nditer(a[0,:].toarray()):
    if i != 0:
        print("one")
a[0,:].toarray().argmax()

one
one


2

* Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.

In [7]:
from scipy.sparse import hstack
X = hstack([X, a])

## 3. Обучите гребневую регрессию с параметрами alpha=1 и random_state=241. Целевая переменная записана в столбце SalaryNormalized.

In [8]:
from sklearn.linear_model import Ridge
y = data['SalaryNormalized']
clf = Ridge(alpha=1, random_state=241)
clf.fit(X, y)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=241, solver='auto', tol=0.001)

## 4. Постройте прогнозы для двух примеров из файла salary-test-mini.csv. Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.

In [9]:
test_data = pd.read_csv('./data/salary-test-mini.csv')
test_data['FullDescription'] = test_data['FullDescription'].str.lower()
test_data['LocationNormalized'] = test_data['LocationNormalized'].str.lower()
test_data['ContractTime'] = test_data['ContractTime'].str.lower()
test_data

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,we currently have a vacancy for an hr project ...,milton keynes,contract,NaN
1,a web developer opportunity has arisen with an...,manchester,permanent,NaN


In [22]:
x = vectorizer.transform(test_data['FullDescription'])
a = enc.transform(test_data[['LocationNormalized', 'ContractTime']].to_dict('records'))
x = hstack([x, a])
x

<2x24627 sparse matrix of type '<class 'numpy.float64'>'
	with 304 stored elements in COOrdinate format>

In [24]:
clf.predict(x)

array([56555.61500155, 37188.32442618])

In [25]:
round(56555.61500155, 2)

56555.62

In [27]:
round(37188.32442618, 2)

37188.32